# Detecting Observer Patterns

### Observer

https://refactoring.guru/design-patterns/observer



In [1]:
import javalang
import os
from glob import glob

path = "./Design-Patterns/**/*.java"

javafiles = list(glob(path, recursive=True))

len(javafiles)

144

In [3]:
def getAST(path: str):
    with open(path, "r") as f:
        code = f.read()
    return javalang.parse.parse(code)

### Publisher with a container

It is necessary for a publisher to maintain a container of subscribers.

In [11]:
def has_collection(node):
    collections = []
    for afield in node.fields:
        if isinstance(afield.type, javalang.tree.ReferenceType):
            if afield.type.name.endswith("List") or afield.type.name.endswith("Set"):
                for decl in afield.declarators:
                    collections.append(decl.name)
    return collections

def has_collection_access(body, collections):
    for path, node in javalang.ast.walk_tree(body):
        if isinstance(node, javalang.tree.MethodInvocation):
            if node.qualifier in collections:
                return True
    return False

def has_adder_remove(node, collections):
    candidates = {}
    for amethod in node.methods:
        if amethod.body is not None:
            if has_collection_access(amethod.body, collections):
                candidates[amethod.name] = amethod
    
    counter = {}
    for aname, amethod in candidates.items():
        if len(amethod.parameters) == 1:
            if isinstance(amethod.parameters[0].type, javalang.tree.ReferenceType):
                if amethod.parameters[0].type.name in counter.keys():
                    counter[amethod.parameters[0].type.name].append(aname)
                else:
                    counter[amethod.parameters[0].type.name] = [aname]
    
    finals = {}
    for atype, methods in counter.items():
        if len(methods) > 1:
            print(atype, methods)
            finals[atype] = methods
    
    return finals
            

pub_candidates = {}

for javafile in javafiles:
    myast = getAST(javafile)
    for path, node in javalang.ast.walk_tree(myast):
        if isinstance(node, javalang.tree.ClassDeclaration):
            colls = has_collection(node)
            if len(colls) > 0:
#                 print("Has collections:", node.name)
                subscriber_types = has_adder_remove(node, colls)
                if len(subscriber_types) > 0:
                    print("Has adder and remover:", node.name)
                    pub_candidates[node.name] = node

                
def has_collection_access(node, collections):
    candidates = {}
    for amethod in node.methods:
        if amethod.body is not None:
            if has_collection_access(amethod.body, collections):
                candidates[amethod.name] = amethod
                
subscribers = []

for javafile in javafiles:
    myast = getAST(javafile)
    for path, node in javalang.ast.walk_tree(myast):
        if isinstance(node, javalang.tree.ClassDeclaration):
            if node.name in subscriber_types.keys():
                subscribers.append(node)
    
mnames = []

for asub in subscribers:
    for amethod in asub.methods:
        mnames.append(amethod.name)
        
for cname, node in pub_candidates.items():
    for amethod in node.methods:
        if len(amethod.parameters) == 0:
            if amethod.body is not None:
                for path, mnode in javalang.ast.walk_tree(amethod.body):
                    if isinstance(mnode, javalang.tree.MethodInvocation):
                        if mnode.member in mnames:
                            print("Publisher:", cname)

IObserver ['addObserver', 'removeObserver']
Has adder and remover: WeatherStation
MenuComponent ['add', 'remove']
Has adder and remover: Menu
Publisher: Menu
Publisher: Menu
Publisher: Menu


### Subscriber as an interface or abstract class

Subscribers should inherit an interface or abstract class to be subscribed.